In [1]:
import datetime
import requests
import pandas as pd
import hopsworks
import datetime
from pathlib import Path
from functions import util
import json
import re
import os
import warnings
warnings.filterwarnings("ignore")

In [2]:
csv_file="./meizhou_data.csv"
util.check_file_path(csv_file)

File successfully found at the path: ./meizhou_data.csv


In [3]:
country="China"
city="Meizhou"
street="Sanjiao town"
aqicn_url="https://api.waqi.info/feed/@4341"

latitude="24.2886"
longitude="116.1226"
today=datetime.date.today()

In [4]:
aqi_api_key_file = './API-key.txt'
util.check_file_path(aqi_api_key_file)

with open(aqi_api_key_file, 'r') as file:
    AQI_API_KEY = file.read().rstrip()

File successfully found at the path: ./API-key.txt


In [5]:
with open('./hopsworks_API.txt', 'r') as file:
    os.environ["HOPSWORKS_API_KEY"] = file.read().rstrip()
    
project = hopsworks.login()

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1171609


In [6]:
secrets = util.secrets_api(project.name)
try:
    secrets.create_secret("AQI_API_KEY", AQI_API_KEY)
except hopsworks.RestAPIError:
    AQI_API_KEY = secrets.get_secret("AQI_API_KEY").value

Connected. Call `.close()` to terminate connection gracefully.


In [7]:
try:
    aq_today_df = util.get_pm25(aqicn_url, country, city, street, today, AQI_API_KEY)
except hopsworks.RestAPIError:
    print("It looks like the AQI_API_KEY doesn't work for your sensor. Is the API key correct? Is the sensor URL correct?")

aq_today_df.head()

,pm25,country,city,street,date,url
0,70.0,China,Meizhou,Sanjiao town,2024-11-15,https://api.waqi.info/feed/@4341


In [8]:
df = pd.read_csv(csv_file,  parse_dates=['date'], skipinitialspace=True)
df

,date,pm25,pm10,o3,no2,so2,co
0,2024-11-01,57.0,24.0,35.0,9.0,2.0,4.0
1,2024-11-02,47.0,38.0,39.0,12.0,9.0,4.0
2,2024-11-03,75.0,35.0,35.0,11.0,4.0,4.0
3,2024-11-04,82.0,34.0,29.0,8.0,4.0,3.0
4,2024-11-05,77.0,39.0,45.0,8.0,3.0,3.0
...,...,...,...,...,...,...,...
3949,2016-01-03,NaN,61.0,16.0,18.0,4.0,13.0
3950,2014-12-31,NaN,60.0,36.0,18.0,8.0,9.0
3951,2014-09-27,NaN,65.0,45.0,9.0,10.0,9.0
3952,2013-12-31,NaN,92.0,35.0,25.0,8.0,16.0


In [9]:
# These commands will succeed if your CSV file didn't have a `median` or `timestamp` column
df = df.rename(columns={"median": "pm25"})
df = df.rename(columns={"timestamp": "date"})

df_aq = df[['date', 'pm25']]
df_aq['pm25'] = df_aq['pm25'].astype('float32')

df_aq

,date,pm25
0,2024-11-01,57.0
1,2024-11-02,47.0
2,2024-11-03,75.0
3,2024-11-04,82.0
4,2024-11-05,77.0
...,...,...
3949,2016-01-03,NaN
3950,2014-12-31,NaN
3951,2014-09-27,NaN
3952,2013-12-31,NaN


In [10]:
df_aq.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3954 entries, 0 to 3953
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   date    3954 non-null   datetime64[ns]
 1   pm25    3941 non-null   float32       
dtypes: datetime64[ns](1), float32(1)
memory usage: 46.5 KB


In [11]:
df_aq.dropna(inplace=True)
df_aq

,date,pm25
0,2024-11-01,57.0
1,2024-11-02,47.0
2,2024-11-03,75.0
3,2024-11-04,82.0
4,2024-11-05,77.0
...,...,...
3936,2014-03-27,143.0
3937,2014-03-28,117.0
3938,2014-03-29,94.0
3939,2014-03-30,103.0


In [12]:
# Your sensor may have columns we won't use, so only keep the date and pm25 columns
# If the column names in your DataFrame are different, rename your columns to `date` and `pm25`
df_aq['country']=country
df_aq['city']=city
df_aq['street']=street
df_aq['url']=aqicn_url
df_aq

,date,pm25,country,city,street,url
0,2024-11-01,57.0,China,Meizhou,Sanjiao town,https://api.waqi.info/feed/@4341
1,2024-11-02,47.0,China,Meizhou,Sanjiao town,https://api.waqi.info/feed/@4341
2,2024-11-03,75.0,China,Meizhou,Sanjiao town,https://api.waqi.info/feed/@4341
3,2024-11-04,82.0,China,Meizhou,Sanjiao town,https://api.waqi.info/feed/@4341
4,2024-11-05,77.0,China,Meizhou,Sanjiao town,https://api.waqi.info/feed/@4341
...,...,...,...,...,...,...
3936,2014-03-27,143.0,China,Meizhou,Sanjiao town,https://api.waqi.info/feed/@4341
3937,2014-03-28,117.0,China,Meizhou,Sanjiao town,https://api.waqi.info/feed/@4341
3938,2014-03-29,94.0,China,Meizhou,Sanjiao town,https://api.waqi.info/feed/@4341
3939,2014-03-30,103.0,China,Meizhou,Sanjiao town,https://api.waqi.info/feed/@4341


In [13]:
earliest_aq_date = pd.Series.min(df_aq['date'])
earliest_aq_date = earliest_aq_date.strftime('%Y-%m-%d')
earliest_aq_date

weather_df = util.get_historical_weather(city, earliest_aq_date, str(today), latitude, longitude)

Coordinates 24.288223266601562°N 116.0702896118164°E
Elevation 85.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s


In [14]:
weather_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3970 entries, 0 to 3969
Data columns (total 6 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   date                         3970 non-null   datetime64[ns]
 1   temperature_2m_mean          3970 non-null   float32       
 2   precipitation_sum            3970 non-null   float32       
 3   wind_speed_10m_max           3970 non-null   float32       
 4   wind_direction_10m_dominant  3970 non-null   float32       
 5   city                         3970 non-null   object        
dtypes: datetime64[ns](1), float32(4), object(1)
memory usage: 155.1+ KB


In [15]:
import great_expectations as ge
aq_expectation_suite = ge.core.ExpectationSuite(
    expectation_suite_name="aq_expectation_suite"
)

aq_expectation_suite.add_expectation(
    ge.core.ExpectationConfiguration(
        expectation_type="expect_column_min_to_be_between",
        kwargs={
            "column":"pm25",
            "min_value":-0.1,
            "max_value":500.0,
            "strict_min":True
        }
    )
)

{"expectation_type": "expect_column_min_to_be_between", "kwargs": {"column": "pm25", "min_value": -0.1, "max_value": 500.0, "strict_min": true}, "meta": {}}

In [16]:
import great_expectations as ge
weather_expectation_suite = ge.core.ExpectationSuite(
    expectation_suite_name="weather_expectation_suite"
)

def expect_greater_than_zero(col):
    weather_expectation_suite.add_expectation(
        ge.core.ExpectationConfiguration(
            expectation_type="expect_column_min_to_be_between",
            kwargs={
                "column":col,
                "min_value":-0.1,
                "max_value":1000.0,
                "strict_min":True
            }
        )
    )
expect_greater_than_zero("precipitation_sum")
expect_greater_than_zero("wind_speed_10m_max")

In [17]:
fs = project.get_feature_store() 

Connected. Call `.close()` to terminate connection gracefully.


In [18]:
dict_obj = {
    "country": country,
    "city": city,
    "street": street,
    "aqicn_url": aqicn_url,
    "latitude": latitude,
    "longitude": longitude
}

# Convert the dictionary to a JSON string
str_dict = json.dumps(dict_obj)

try:
    secrets.create_secret("SENSOR_LOCATION_JSON", str_dict)
except hopsworks.RestAPIError:
    print("SENSOR_LOCATION_JSON already exists. To update, delete the secret in the UI (https://c.app.hopsworks.ai/account/secrets) and re-run this cell.")
    existing_key = secrets.get_secret("SENSOR_LOCATION_JSON").value
    print(f"{existing_key}")

Secret created successfully, explore it at https://c.app.hopsworks.ai:443/account/secrets


In [19]:
air_quality_fg = fs.get_or_create_feature_group(
    name='air_quality',
    description='Air Quality characteristics of each day',
    version=1,
    primary_key=['city', 'street', 'date'],
    event_time="date",
    expectation_suite=aq_expectation_suite
)

In [20]:
air_quality_fg.insert(df_aq)

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/1171609/fs/1162312/fg/1348992
2024-11-15 16:23:58,560 INFO: 	1 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1171609/fs/1162312/fg/1348992


Uploading Dataframe: 0.00% |          | Rows 0/3941 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: air_quality_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/1171609/jobs/named/air_quality_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x2701bcb2680>,
 {
   "success": true,
   "results": [
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
         "kwargs": {
           "column": "pm25",
           "min_value": -0.1,
           "max_value": 500.0,
           "strict_min": true
         },
         "meta": {
           "expectationId": 672813
         }
       },
       "result": {
         "observed_value": 5.0,
         "element_count": 3941,
         "missing_count": null,
         "missing_percent": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2024-11-15T03:23:58.000560Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       }
     }
   ],
   "evaluation_parameters": {},
   "statistics": {
     "evaluated_expectations": 1,
     "successful_expectations": 1,
     "un

In [21]:
air_quality_fg.update_feature_description("date", "Date of measurement of air quality")
air_quality_fg.update_feature_description("country", "Country where the air quality was measured (sometimes a city in acqcn.org)")
air_quality_fg.update_feature_description("city", "City where the air quality was measured")
air_quality_fg.update_feature_description("street", "Street in the city where the air quality was measured")
air_quality_fg.update_feature_description("pm25", "Particles less than 2.5 micrometers in diameter (fine particles) pose health risk")

In [22]:
# Get or create feature group 
weather_fg = fs.get_or_create_feature_group(
    name='weather',
    description='Weather characteristics of each day',
    version=1,
    primary_key=['city', 'date'],
    event_time="date",
    expectation_suite=weather_expectation_suite
) 

In [23]:
# Insert data
weather_fg.insert(weather_df)

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/1171609/fs/1162312/fg/1348993
2024-11-15 16:27:55,248 INFO: 	2 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1171609/fs/1162312/fg/1348993


Uploading Dataframe: 0.00% |          | Rows 0/3970 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: weather_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/1171609/jobs/named/weather_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x2701bc4f8e0>,
 {
   "success": true,
   "results": [
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
         "kwargs": {
           "column": "wind_speed_10m_max",
           "min_value": -0.1,
           "max_value": 1000.0,
           "strict_min": true
         },
         "meta": {
           "expectationId": 672815
         }
       },
       "result": {
         "observed_value": 3.096837043762207,
         "element_count": 3970,
         "missing_count": null,
         "missing_percent": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2024-11-15T03:27:55.000248Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       }
     },
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_colum

In [24]:
weather_fg.update_feature_description("date", "Date of measurement of weather")
weather_fg.update_feature_description("city", "City where weather is measured/forecast for")
weather_fg.update_feature_description("temperature_2m_mean", "Temperature in Celsius")
weather_fg.update_feature_description("precipitation_sum", "Precipitation (rain/snow) in mm")
weather_fg.update_feature_description("wind_speed_10m_max", "Wind speed at 10m abouve ground")
weather_fg.update_feature_description("wind_direction_10m_dominant", "Dominant Wind direction over the dayd")